In [1]:
!pip install dnspython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 6.5 MB/s eta 0:00:00


In [3]:
import dns.resolver
import socket

def extract_ptr_records(ip):
    """Extract PTR records for a given IP address."""
    try:
        reversed_ip = '.'.join(reversed(ip.split('.'))) + '.in-addr.arpa'
        answers = dns.resolver.resolve(reversed_ip, 'PTR')
        
        if not answers:
            print("No PTR records found for this IP address.")
            return

        print("PTR Records Found:")
        for i, rdata in enumerate(answers, start=1):
            print(f"{i}. {rdata}")

    except dns.resolver.NoAnswer:
        print("No PTR records found for this IP address.")
    except dns.resolver.NXDOMAIN:
        print("Domain does not exist.")
    except Exception as e:
        print(f"Error: {e}")

def get_ip_address(domain):
    """Resolve a domain to an IP address."""
    try:
        return socket.gethostbyname(domain)
    except socket.gaierror:
        print(f"Error: Could not resolve the domain '{domain}'.")
        return None

def validate_ip(ip):
    """Validate the IP address format."""
    parts = ip.split('.')
    return len(parts) == 4 and all(part.isdigit() and 0 <= int(part) <= 255 for part in parts)

def validate_domain(domain):
    """Basic validation for domain names."""
    if not domain or len(domain) > 255:
        return False
    if domain[-1] == '.':  # Allow trailing dot
        domain = domain[:-1]
    return all(0 < len(label) < 64 and label.isalnum() for label in domain.split("."))

def menu():
    """Display a menu and process user choices."""
    print("Welcome to the PTR Record Extractor")
    while True:
        print("\n1 - Input IP")
        print("2 - Input Domain Name")
        print("3 - Exit")
        choice = input("Enter your choice: ").strip()

        if choice == "1":
            ip = input("Enter the IP address to query (e.g., 75.2.70.75): ").strip()
            if validate_ip(ip):
                extract_ptr_records(ip)
            else:
                print("Invalid IP address format. Please try again.")
        elif choice == "2":
            domain = input("Enter the domain to query (e.g., example.com): ").strip()
            if validate_domain(domain):
                ip_address = get_ip_address(domain)
                if ip_address:
                    print(f"The IP address of '{domain}' is {ip_address}.")
                    extract_ptr_records(ip_address)
            else:
                print("Invalid domain name format. Please try again.")
        elif choice == "3":
            print("Exiting program.")
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

if __name__ == "_main_":
    menu()